Продолжим работу с данными, которые были использованы в ДЗ 2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [1]:
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn import metrics
import seaborn as sns
import warnings
from tqdm import tqdm
from typing import List, Tuple
from scipy.stats import ttest_rel
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from classes_and_functions import bootstrap_calculate_confidence_interval, make_cross_validation,make_cross_validation_gr

warnings.simplefilter("ignore")
from sklearn.model_selection import TimeSeriesSplit
%matplotlib inline

In [3]:
train= pd.read_csv('train.csv')

In [4]:
train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,3166995,0,3958217,39.00,W,1877,310.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179996,3166996,0,3958237,59.95,W,10075,514.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179997,3166997,0,3958241,34.00,W,6053,122.0,150.0,mastercard,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179998,3166998,0,3958260,59.00,W,7726,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train.set_index('TransactionID',inplace=True)

In [6]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 2987000 to 3166999
Columns: 393 entries, isFraud to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 541.1+ MB


In [8]:
object_features = train.select_dtypes(include=[np.object]).columns
train[object_features]= train[object_features].astype('category')

In [9]:
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])

In [10]:
scors = pd.DataFrame()

### Базовая модель

In [11]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    'num_leaves':45,
    'max_depth':4,
    'learning_rate':0.1,
    'n_estimators':5000,
    "metric": "auc",
    "n_jobs": 15,
    'reg_alpha':2,
    'reg_lambda':2,
    "random_state": 30,
#     'device':"gpu"
}

In [12]:
model = lgb.LGBMClassifier(**params)

In [13]:
%%time
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['base','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 31.5 s


(1967,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9965549735307786)]),
              'valid_1': OrderedDict([('auc', 0.9527215435494588)])}))

In [14]:
# train.TransactionDT =  pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT*10**9)

год, месяц, день недели, час, день.

In [15]:
train['year'] =  (pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT*10**9)).apply(lambda x: x.year)

In [16]:
train['month'] = (pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT*10**9)).apply(lambda x: x.month)

In [17]:
train['day'] =  (pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT*10**9)).apply(lambda x: x.day)

In [18]:
train['hour'] =  (pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT*10**9)).apply(lambda x: x.hour)

In [19]:
train['weekday'] =  (pd.to_datetime('2017-12-01') + pd.to_timedelta(train.TransactionDT*10**9)).apply(lambda x: x.weekday())

In [20]:
train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V335,V336,V337,V338,V339,year,month,day,hour,weekday
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5
2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5
2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5
2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,2017,12,2,0,5
2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,2017,12,2,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166995,0,3958217,39.00,W,1877,310.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,2018,1,15,19,0
3166996,0,3958237,59.95,W,10075,514.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,2018,1,15,19,0
3166997,0,3958241,34.00,W,6053,122.0,150.0,mastercard,195.0,debit,...,NaN,NaN,NaN,NaN,NaN,2018,1,15,19,0


In [21]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['TransactionDT','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 22.9 s


(1310,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9924609097714104)]),
              'valid_1': OrderedDict([('auc', 0.9483256699856828)])}))

Скор на валидации упал

### Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

In [22]:
train['card1_card2'] = train.apply(lambda x: str(x.card1)+'_' + str(x.card2),axis=1)

In [23]:
train['card1_card2_card_3_card_5'] =\
    train.apply(lambda x: str(x.card1_card2)+'_' + str(x.card3) + '_'+str(x.card4) + '_'+ str(x.card5) ,axis=1)

In [24]:
train['card1_card2_card_3_card_5_addr1_addr2'] =\
    train.apply(lambda x: str(x.card1_card2_card_3_card_5)+'_' + str(x.addr1) + '_'+str(x.addr2) ,axis=1)

In [25]:
object_features = object_features.to_list()
object_features = object_features + ['card1_card2','card1_card2_card_3_card_5','card1_card2_card_3_card_5_addr1_addr2']

In [26]:
train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V338,V339,year,month,day,hour,weekday,card1_card2,card1_card2_card_3_card_5,card1_card2_card_3_card_5_addr1_addr2
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,2017,12,2,0,5,13926_nan,13926_nan_150.0_discover_142.0,13926_nan_150.0_discover_142.0_315.0_87.0
2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,2017,12,2,0,5,2755_404.0,2755_404.0_150.0_mastercard_102.0,2755_404.0_150.0_mastercard_102.0_325.0_87.0
2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,2017,12,2,0,5,4663_490.0,4663_490.0_150.0_visa_166.0,4663_490.0_150.0_visa_166.0_330.0_87.0
2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,2017,12,2,0,5,18132_567.0,18132_567.0_150.0_mastercard_117.0,18132_567.0_150.0_mastercard_117.0_476.0_87.0
2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,2017,12,2,0,5,4497_514.0,4497_514.0_150.0_mastercard_102.0,4497_514.0_150.0_mastercard_102.0_420.0_87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166995,0,3958217,39.00,W,1877,310.0,150.0,mastercard,224.0,debit,...,NaN,NaN,2018,1,15,19,0,1877_310.0,1877_310.0_150.0_mastercard_224.0,1877_310.0_150.0_mastercard_224.0_272.0_87.0
3166996,0,3958237,59.95,W,10075,514.0,150.0,mastercard,224.0,debit,...,NaN,NaN,2018,1,15,19,0,10075_514.0,10075_514.0_150.0_mastercard_224.0,10075_514.0_150.0_mastercard_224.0_315.0_87.0
3166997,0,3958241,34.00,W,6053,122.0,150.0,mastercard,195.0,debit,...,NaN,NaN,2018,1,15,19,0,6053_122.0,6053_122.0_150.0_mastercard_195.0,6053_122.0_150.0_mastercard_195.0_330.0_87.0


In [27]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['concate cards','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 25.8 s


(1379,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.9980582343918935)]),
              'valid_1': OrderedDict([('auc', 0.9490802202629252)])}))

скор на валидации почти не изменился с предидушим

#### Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [28]:
train.card1 = train.card1.map(train.card1.value_counts())

In [29]:
train.card1.value_counts()

4376    4376
3832    3832
3393    3393
3170    3170
1       3090
        ... 
105      105
90        90
85        85
82        82
80        80
Name: card1, Length: 302, dtype: int64

In [30]:
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
    train[i] = train[i].map(train[i].value_counts())

In [31]:
%%time
train[object_features]= train[object_features].astype(str)
train[object_features]= train[object_features].astype('category')
x_train,x_valid,y_train,y_valid =\
train_test_split(train.drop('isFraud',axis=1),train['isFraud'], test_size = 0.2,random_state=30,stratify =train['isFraud'])
model.fit(x_train,y_train,early_stopping_rounds  = 50,\
            eval_set =[(x_train,y_train),(x_valid,y_valid)],verbose = False)
scors.loc['FrequencyEncoder','auc'] = model.best_score_['valid_1']['auc']
model.best_iteration_,model.best_score_

Wall time: 22.7 s


(1171,
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('auc', 0.997886311039727)]),
              'valid_1': OrderedDict([('auc', 0.9490467582515151)])}))

### Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [32]:
train.TransactionAmt

TransactionID
2987000     68.50
2987001     29.00
2987002     59.00
2987003     50.00
2987004     50.00
            ...  
3166995     39.00
3166996     59.95
3166997     34.00
3166998     59.00
3166999    226.00
Name: TransactionAmt, Length: 180000, dtype: float64

In [ ]:
for i in [ 'card1', 'card2', 'card3','card5' ,'card6', 'addr1', 'addr2']:
stats = 